In [1]:
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [1]:
import torch 
import pandas as pd
import os
import numpy 
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

import random
import time
import datetime

In [3]:
# data load

import pandas as pd
bnk = pd.read_excel(r'/home/kelly790/.1/esg_data/금융/BNK금융.xlsx')
kb = pd.read_excel(r'/home/kelly790/.1/esg_data/금융/KB금융.xlsx')
ibk = pd.read_excel(r'/home/kelly790/.1/esg_data/금융/기업은행.xlsx')
shinhan = pd.read_excel(r'/home/kelly790/.1/esg_data/금융/신한금융.xlsx')
woori = pd.read_excel(r'/home/kelly790/.1/esg_data/금융/우리금융.xlsx')
hana = pd.read_excel(r'/home/kelly790/.1/esg_data/금융/하나금융.xlsx')


kia = pd.read_excel(r'/home/kelly790/.1/esg_data/자동차/기아자동차.xlsx')
hyundae = pd.read_excel(r'/home/kelly790/.1/esg_data/자동차/현대자동차.xlsx')

lg = pd.read_excel(r'/home/kelly790/.1/esg_data/전자/LG.xlsx')
samsung = pd.read_excel(r'/home/kelly790/.1/esg_data/전자/Samsung.xlsx')

dusan = pd.read_excel(r'/home/kelly790/.1/esg_data/중공업/두산중공업.xlsx')
samsung_in = pd.read_excel(r'/home/kelly790/.1/esg_data/중공업/삼성중공업.xlsx')
hyundae_in = pd.read_excel(r'/home/kelly790/.1/esg_data/중공업/현대중공업.xlsx')

daehan = pd.read_excel(r'/home/kelly790/.1/esg_data/항공/대한항공_원본.xlsx')
asiana = pd.read_excel(r'/home/kelly790/.1/esg_data/항공/아시아나항공_원본.xlsx')

finance = bnk.append([kb,ibk,shinhan,woori,hana, kia,hyundae, lg,samsung, dusan,samsung_in, hyundae_in,daehan,asiana])

fin_e = finance.loc[finance['label'] == 'E']
fin_s = finance.loc[finance['label'] == 'S']
fin_g = finance.loc[finance['label'] == 'G']
fin_o = finance.loc[finance['label'] == 'O']
fin_n = finance[finance['label'].str.contains('N',na=True)]
fin_n['label'] = 'N'
fin = fin_e.append([fin_g,fin_s, fin_o,fin_n], sort=False)

/tmp/ipykernel_4119865/1931665312.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finance = bnk.append([kb,ibk,shinhan,woori,hana, kia,hyundae, lg,samsung, dusan,samsung_in, hyundae_in,daehan,asiana])
/tmp/ipykernel_4119865/1931665312.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_n['label'] = 'N'
/tmp/ipykernel_4119865/1931665312.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fin = fin_e.append([fin_g,fin_s, fin_o,fin_n], sort=False)


In [4]:
fin = fin.drop(['발행년도','sentence'],axis=1)
fin = fin.reset_index()
fin = fin.drop([fin.columns[0]], axis=1)
fin

,label,text
0,E,한편 장기적으로 국제 사회에 큰 위협이 될 것으로 예상되는 기후변화에 대한 대응체계도 강화해 나가고 있습니다.
1,E,기후변화의 규제 강화에 따라 투자사업들에 대한 리스크 익스포저의 변화 투자 수익률의 감소 등이 예상되는 가운데 BNK금융그룹은 저탄소 경제로의 이행 및 녹색 금융 서비스 에 대한 소비자 수요 증가에 대비하여 환경경영 강화 친환경 금융 확대 PFProject Financing 실행시 환경영향평가 시행 등 다양한 환경분야 리스크 관리 기법을 적용하는 중...
2,E,그리고 이 를 통해 금융을 통한 환경보호 및 기후변화 대응을 실현하기 위한 토대를 구축해 나가고 있습니다.
3,E,BNK금융그룹은 금융산업의 미래지향적 친환경 경영을 선도하는 환경경영 전략을 추진하고 있습 니다.
4,E,너지 절감 활동을 추진함으로써 효율적으로 환경경영을 실천하고 있습니다.
...,...,...
101088,N,금호아시아나그룹의 경영이념인 세계 일류의 가치창출 기업 종업원과 함께 하는 기업 이웃으로부터 사랑받는 기업을 창조하기 위해 그룹의 임직원 각자 가 지켜야 할 바람직한 행동과 가치판단의 기준을 정립하고자 다음과 같은 윤리강령을 선포한다.
101089,N,윤리경영 교육 실시 전 임직원을 대상으로 다양한 윤리경영 교육을 통해 윤리경영에 대한 인식 확산 및 공감대 형성을 유도하고 있으며 사이버 윤리경영 교육을 비롯한 직무교육과 신임 임원 신입사원 직급별 승격자 교육 시 윤리경영 교육을 실시하고 있습니다.
101090,N,금품 및 선물 안 받기 캠페인 실시 아시아나항공은 윤리경영의 정착을 위하여 매년 협력회사로부터 금품 및 선물 수수 금지를 내용으로 하는 자율 캠페 인을 실시하고 있습니다.
101091,N,아시아나 환경 스쿨 지역구 초중학생을 대상으로 아시아나 환경 스쿨을 정기적으로 개최하여 환경 현장체험과 관련 지식을 전파하고 있습니다.


In [5]:
fin = fin.drop(labels=range(12643,13467), axis=0)
fin.reset_index(inplace=True)
fin = fin.drop([fin.columns[0]],axis=1)

In [6]:
fin = fin.drop(labels=range(21375,21630), axis=0)
fin.reset_index(inplace=True)
fin = fin.drop([fin.columns[0]],axis=1)

In [7]:
fin = fin.drop(labels=range(52674,54632), axis=0)
fin.reset_index(inplace=True)
fin = fin.drop([fin.columns[0]],axis=1)

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(fin['label'])
fin['label'] = encoder.transform(fin['label'])

In [9]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'E', 1: 'G', 2: 'N', 3: 'O', 4: 'S'}

In [3]:
device = torch.device('cuda:1')
device

device(type='cuda', index=1)

In [2]:
torch.cuda.is_available()

False

In [11]:
# 훈련셋과 테스트셋 데이터 로드
from sklearn.model_selection import train_test_split

train, test = train_test_split(fin, test_size=0.2, random_state=42)

# 문장 추출
sentences = train['text']

# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 추출
labels = train['label'].values

# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=42, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 문장 추출
sentences = test['text']

# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 추출
labels = test['label'].values

# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 5

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [12]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경

In [13]:
from tqdm import tqdm

# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in tqdm(range(0, epochs)):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in tqdm(enumerate(train_dataloader)):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch    
        
        b_input_ids = b_input_ids.long()
        b_input_mask = b_input_mask.long()
        b_labels = b_labels.long()
        
        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in tqdm(validation_dataloader):
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

  0%|          | 0/5 [00:00<?, ?it/s]


======== Epoch 1 / 5 ========
Training...


  Batch   500  of  2,207.    Elapsed: None.


  Batch 1,000  of  2,207.    Elapsed: None.


  Batch 1,500  of  2,207.    Elapsed: None.


...

  Batch 2,000  of  2,207.    Elapsed: None.


2207it [05:26,  6.75it/s]



  Average training loss: 0.232
  Training epcoh took: None

Running Validation...


100%|██████████| 5/5 [27:57<00:00, 335.52s/it]

  Accuracy: 0.817
  Validation took: None

Training complete!


In [15]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in tqdm(enumerate(test_dataloader)):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

105it [00:03, 27.48it/s]

  Batch   100  of    613.    Elapsed: None.


204it [00:07, 27.45it/s]

  Batch   200  of    613.    Elapsed: None.


303it [00:11, 27.40it/s]

...

  Batch   500  of    613.    Elapsed: None.


603it [00:22, 27.15it/s]

  Batch   600  of    613.    Elapsed: None.


613it [00:22, 27.35it/s]


Accuracy: 0.814
Test took: None


In [16]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks